In [ ]:
# Import libraries
from monai.data import DataLoader
from monai.transforms import (EnsureChannelFirstd,
Compose, LoadImaged, ResampleToMatchd, MapTransform, SaveImaged, LoadImage)

from monai.apps import TciaDataset
from monai.apps.auto3dseg import AutoRunner
from monai.bundle import ConfigParser

from monai.config import print_config
import json

print_config()

In [ ]:
# Add a fold key to all the training data
train_dataset.datalist = [{**item, 'fold': 0} for item in train_dataset.datalist]

# Change "seg" to "label" in the datalist
for item in train_dataset.datalist:
    item["label"] = item.pop("seg")
    
for item in test_dataset.datalist:
    item["label"] = item.pop("seg")

# Concatenate the training and test datalists
data_list = {"training": train_dataset.datalist, "testing": test_dataset.datalist}

datalist_file = "../auto3dseg_datalist.json"
with open(datalist_file, "w") as f:
    json.dump(data_list, f)
# Create input configuration .yaml file
input_config = {
    "name": "HCC-TACE-Seg",
    "task": "segmentation",  
    "modality": "CT", 
    "datalist": "../auto3dseg_datalist.json", 
    "dataroot": "../data", 
}

config_yaml = "../auto3dseg_config.yaml"
ConfigParser.export_config_file(input_config, config_yaml)

In [ ]:
runner = AutoRunner(work_dir = "../data/auto3dseg",train=False, input=input_config, algos=["segresnet", "segresnet2d", "swinunetr"])
runner.run()